In [ ]:
from transformers import AutoModelForCausalLM, OPTForCausalLM, AutoTokenizer,AutoModelForSeq2SeqLM,TrainingArguments,Trainer,DataCollatorForLanguageModeling,TextDataset
from peft import LoraConfig,TaskType,get_peft_model
from datasets import load_dataset
import json

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r = 32,
    lora_alpha=32,
    lora_dropout=0.1
)
model_id = 'openai-community/gpt2'
training_args = TrainingArguments (
    output_dir='output/test',
    learning_rate=1e-3,
    per_device_eval_batch_size=32,
    per_device_train_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy='no',
    save_strategy='epoch',
    #load_best_model_at_end='True',
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id)
model = get_peft_model(model,peft_config)
model.print_trainable_parameters()
tokenizer = AutoTokenizer.from_pretrained(model_id)
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=False)


In [ ]:
def preprocess(input_path = 'intents.json'):
    with open(input_path,'r') as file:
        data = json.load(file)
    preprocess_data = []
    for intent in data['intents']:
        for patternn in intent['patterns']:
            preprocess_data.append(f'User: {patternn}\n')
            for response in intent['responses']:
                preprocess_data.append(f'Assistant: {response}\n')
    return ''.join(preprocess_data)
def save_preprocess(data,output_path = 'data/data.txt'):
    with open(output_path,'w') as file:
        file.write(data)

In [ ]:
data = preprocess()
save_preprocess(data)

In [ ]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='data/data.txt',
    block_size=64
)

In [ ]:
print(train_dataset[64])

In [ ]:
trainer = Trainer(
    model = model,
    args=  training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=collator
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("output_dir")

In [ ]:
a_model = AutoModelForCausalLM.from_pretrained("output_dir")